## Intermediate Lab:  Creating A Dataset Using the GitHub API

In this lab you'll create a dataset containing all the meta information about your classmates github repos, using only the csv file with everyone's url.  

The process will be done in these 4 general steps:

 - load in the csv file
 - clean the individual lines of each file to get them ready to use
 - connect to the GitHub api to obtain information about everyone's repos
 - re-structure this information to turn it into a dataframe.
 
It'll be a great way to work through the first step of many data science problems: creating a workable dataset out of unorganized, messy data.  Let's get started!

### Step 1:  Load in the csv file with everyone's github repo

**Note:** There are a number of ways to do this, but the easiest way is usually this:

    `with open('file.csv') as f:

        data = f.readlines()`

In [70]:
import requests
import pandas as pd

In [71]:
with open('datrepos.csv') as file:
    data = file.readlines()

What you should have now is a list, and each item is a string that contains the comma separated values of each cell in the row of that csv file.  

It should generically look like this:

    `['Name,Repo\n',
      'Person 1,https://github.com/username1\n',
      'Person 2,https://github.com/username2\n',
       ......
       ]`

Double check this is the case.

In [72]:
data

['Name,Github URL\n',
 'Jonathan,https://github.com/JonathanBechtel\n',
 'Rezwana,https://github.com/rezsharmeen\n',
 'Marnie,https://github.com/marnierl\n',
 'Kristina,https://github.com/hayniek\n',
 'Harley,https://github.com/harleyhoffmann\n',
 'Uma,https://github.com/umap1230\n',
 'Andrew Cal,https://github.com/AndrewCal2013\n',
 'Zarina ,https://github.com/zarinajm7\n',
 'Lisa,https://github.com/lisastaal\n',
 'Emma ,https://github.com/ewynn5\n',
 'Jacob,https://github.com/jdonahue94\n',
 'Tina,https://github.com/tinagads\n',
 'Danielle,https://github.com/dlemi\n',
 ',\n',
 ',\n',
 'Jake H,https://github.com/jhoernsch\n',
 'Avinash,https://github.com/avirathore2\n',
 ',\n',
 ',https://github.com/bebenono/sample-ga-repo\n',
 'Krithi N,https://github.com/nkrithi\n']

The only thing we need out of each item is the persons username, that part contained in the string at: `https://github.com/username_here`.  Everything else is junk.  

We'll need to go through a few steps to get our info down to a usable format.  

### Step 2: Remove the `\n` from each item in the list, as well as the item that contains the header info.

When you're done you should have a list that looks like this:

      `[
      'Person 1,https://github.com/username1',
      'Person 2,https://github.com/username2',
       ......
       ]`

**hint:** The `replace()` method for strings is probably one of the more useful options that you have.  If you want to replace something with nothing, you can simply specify `""` for that part.

In [73]:
cleaned_data = [student.strip() for student in data]
cleaned_data.pop(0)
final_data = [student for student in cleaned_data if len(student) > 1]
final_data

['Jonathan,https://github.com/JonathanBechtel',
 'Rezwana,https://github.com/rezsharmeen',
 'Marnie,https://github.com/marnierl',
 'Kristina,https://github.com/hayniek',
 'Harley,https://github.com/harleyhoffmann',
 'Uma,https://github.com/umap1230',
 'Andrew Cal,https://github.com/AndrewCal2013',
 'Zarina ,https://github.com/zarinajm7',
 'Lisa,https://github.com/lisastaal',
 'Emma ,https://github.com/ewynn5',
 'Jacob,https://github.com/jdonahue94',
 'Tina,https://github.com/tinagads',
 'Danielle,https://github.com/dlemi',
 'Jake H,https://github.com/jhoernsch',
 'Avinash,https://github.com/avirathore2',
 ',https://github.com/bebenono/sample-ga-repo',
 'Krithi N,https://github.com/nkrithi']

### Step 3:  Separate the username in each string from everything else

When you're done you should have a new list that looks like this:

    `[
       'username1',
       'username2',
       ...
     ]`

**hint:** `split()` is a helpful method for this.  Since the parts of the string are separated by a `/` you could use that to split the string.

In [74]:
usernames = [student.split('/')[-1] for student in final_data]

In [75]:
usernames

['JonathanBechtel',
 'rezsharmeen',
 'marnierl',
 'hayniek',
 'harleyhoffmann',
 'umap1230',
 'AndrewCal2013',
 'zarinajm7',
 'lisastaal',
 'ewynn5',
 'jdonahue94',
 'tinagads',
 'dlemi',
 'jhoernsch',
 'avirathore2',
 'sample-ga-repo',
 'nkrithi']

### Step 4: Obtain the repo data for every single github username

The repository info for every single public account is available via the following url: `https://api.github.com/users/:the_username/repos`

So basically, `requests.get('https://api.github.com/users/:the_username/repos').json()` will return a list filled with every single repo that user has.  

When you're done, you should have a *list of lists*, with each list containing each users individual repos.  It'll look like this:

`[[{user1, repo1}, {user1, repo2}], [{user2, repo1}], [{user3, repo1}, {user3, repo2}, {user3, rep3}], .....]`

**Warning:** We're using the free, unauthenticated version of the API here.  That means we can only make 60 API calls per hour before getting throttled.  If we've used up our bandwidth the response you'll get is a dictionary telling you we've exceeded our rate limit or something similar.

If that's the case, try using your phone (or your neighbors) as a hotspot and connect from there to get a new IP address.

In [76]:
# looping through our list of usernames
# creating a new list that holds lists of dictionaries
# each sub list holds dictionaries with individual repo data
# [ [{}, {}, {}], [{}, {}, {}], [{}, {}] ]
base_url = 'https://api.github.com'
repo_lists = [requests.get(f"{base_url}/users/{name}/repos").json() for name in usernames]

In [77]:
repo_lists

[[{'id': 260764681,
   'node_id': 'MDEwOlJlcG9zaXRvcnkyNjA3NjQ2ODE=',
   'name': 'bitcoin',
   'full_name': 'JonathanBechtel/bitcoin',
   'private': False,
   'owner': {'login': 'JonathanBechtel',
    'id': 481696,
    'node_id': 'MDQ6VXNlcjQ4MTY5Ng==',
    'avatar_url': 'https://avatars.githubusercontent.com/u/481696?v=4',
    'gravatar_id': '',
    'url': 'https://api.github.com/users/JonathanBechtel',
    'html_url': 'https://github.com/JonathanBechtel',
    'followers_url': 'https://api.github.com/users/JonathanBechtel/followers',
    'following_url': 'https://api.github.com/users/JonathanBechtel/following{/other_user}',
    'gists_url': 'https://api.github.com/users/JonathanBechtel/gists{/gist_id}',
    'starred_url': 'https://api.github.com/users/JonathanBechtel/starred{/owner}{/repo}',
    'subscriptions_url': 'https://api.github.com/users/JonathanBechtel/subscriptions',
    'organizations_url': 'https://api.github.com/users/JonathanBechtel/orgs',
    'repos_url': 'https://api.g

### Step 5: Create a 'flat' list that contains every unique repo for every single user

When you're done you should have a list that looks like this: `[{user1 repo1}, {user1 repo2}, ....{user n, repo m}]`

Ie, instead of having a list filled with other lists with dictionaries inside of them, make it a list with just dictionaries on the inside.

Ie, no nested levels like you had before.

So, go from this:

`[[{user1, repo1}, {user1, repo2}, {user1, repo3}], [{user2, repo1}, {user2, repo2}]]`
    
To this:

`[{user1, repo1}, {user1, repo2}, {user1, repo3}, {user2, repo1}, {user2, repo2}]`
    
If you have questions about what this entails, then please contact me ASAP.

In [78]:
repos = [repo for user in repo_lists for repo in user]
repos

[{'id': 260764681,
  'node_id': 'MDEwOlJlcG9zaXRvcnkyNjA3NjQ2ODE=',
  'name': 'bitcoin',
  'full_name': 'JonathanBechtel/bitcoin',
  'private': False,
  'owner': {'login': 'JonathanBechtel',
   'id': 481696,
   'node_id': 'MDQ6VXNlcjQ4MTY5Ng==',
   'avatar_url': 'https://avatars.githubusercontent.com/u/481696?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/JonathanBechtel',
   'html_url': 'https://github.com/JonathanBechtel',
   'followers_url': 'https://api.github.com/users/JonathanBechtel/followers',
   'following_url': 'https://api.github.com/users/JonathanBechtel/following{/other_user}',
   'gists_url': 'https://api.github.com/users/JonathanBechtel/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/JonathanBechtel/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/JonathanBechtel/subscriptions',
   'organizations_url': 'https://api.github.com/users/JonathanBechtel/orgs',
   'repos_url': 'https://api.github.com/users/Jon

### Step 6:  Get information about the name, owner, url, and date of every single repo.

We are **almost** there to getting a true dataset out of our API calls.  The problem we have now is that our dictionary is nested.  Ie, some of the values are other dictionaries or lists.  

To make these values accessible to pandas we need to have them all on an equal footing.

In the dictionary for each repo there are keys called `name`, `login`, `html_url`, and `created_at`.  These are going to populate the values for our different columns.

If you look, you'll see that the `login` key is nested inside another dictionary.

What we want to do is have the values for each of these keys inside their own separate lists, so their values can be re-used more easily.

So what you need to do is use for loops to create a list that contains all the individual values for each of these dictionary keys.  

You should results that look like this:

`names  = [list filled with all of the values of the name key]`

`logins = [list filled with all of the values of the login key]`

`urls   = [list filled with all of the values of the html_url key]`

`dates  = [list filled with all of the values of the created_at key]`

**hint:** Notice that the `login` key is nested inside a dictionary that's the value to the `owner` key at the outer level.

In [79]:
names = [repo['name'] for repo in repos]
owners = [repo['owner']['login'] for repo in repos]
urls = [repo['html_url'] for repo in repos]
dates = [repo['created_at'] for repo in repos]

TypeError: string indices must be integers

### Step 7:  Create a dictionary with the data created from step 7

Your final output should look like this:

    `{
       'Owner': [list with the `login` values for each user],
       'Name' : [list with the `name` values for each user],
       'URL'  : [list with the `html_url` values for each user],
       'Date' : [list with the `created_at` values for each user]
     }`

In [80]:
data_dict = {'Owner': owners,
             'Name': names,
             'URL': urls,
             'Date': dates
}

NameError: name 'owners' is not defined

### Step 8:  Pass your dictionary into the `pd.dataframe()` method to get your final dataset  

Use the `df.head()` method to confirm that you have something that's formatted appropriately.

In [ ]:
# your answer here
